<div style="padding:20px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#196F3D;overflow:hidden;font-weight:500">TPS JUNE 2022</div>
<br>
<br>
<br>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.0 | Introduction</b></p>
</div>

Data Cleaning is the process of finding and correcting the inaccurate/incorrect data that are present in the dataset. One such process needed is to do something about the values that are missing in the dataset. In real life, many datasets will have many missing values, so dealing with them is an important step.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import missingno as msno
from sklearn.impute import SimpleImputer

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.0 | Load data and basic checks</b></p>
</div>


In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
print("Data Shape: There are {:,.0f} rows and {:,.0f} columns.\nMissing values = {}, Duplicates = {}.\n".
      format(data.shape[0], data.shape[1],data.isna().sum().sum(), data.duplicated().sum()))
df=data.describe()
display(df.style.format('{:,.3f}')
        .background_gradient(subset=(df.index[1:],df.columns[:]), cmap='GnBu'))


In [ ]:
# get the number of missing data points per column
missing_values_count = data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:30]

Quick insight:
As we can see above, some of the columns have no missing values, which is a good sign. We need to see if any of the columns are correlated.

In [ ]:
# how many total missing values do we have?
total_cells = np.product(data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

## Lets visulize first 500 rows for missing values

In [ ]:
import missingno as msno
msno.matrix(data.sample(500))

Quick insights:
1) The missing data location is vary random. We need to identify is there any pattern to it.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import seaborn as sns
primary_bgcolor = "#f4f0ea"
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
corr = data.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig, ax = plt.subplots(figsize=(90, 50), facecolor=primary_bgcolor)
# ax.text(-1.1, 0.16, 'Correlation between the Continuous Features', fontsize=10, fontweight='bold', fontfamily='serif')
ax.text(-1.1, 0.3, 'There is no features that pass more than 0.32 correlation within each other', fontsize=13, fontweight='light', fontfamily='serif')


# plot heatmap
res=sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',annot_kws={"size": 25},
            cbar_kws={"shrink": .2}, vmin=0, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 28)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 28)
# yticks
plt.yticks(rotation=0)
plt.show()

Quick insight: As we can see above some of the columns are highly correlated. We can use this informaiton to replace the missing data.

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | Imputation techniques</b></p>
</div>
Typically replacing the missing data with new data is called imputation. There are various techniques for this task depeding on the type of the data that we have. Typically imputation can be divided into following types

1) Constant value imputation: As name suggests all the missing values will be replaced with constant value.  <br> 
2) Basic statiscal value imputation: As name suggests missing valeus will be treated as the mean, median or most frequent value of that column  <br>
3) Advanced techniques:  <br>
When basic techniques does not address the problem, machine learning algorithms can be used to identify the substitute value for the missing values. 


<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.1 | Basic statistics- Mean value</b></p>
</div>

In [ ]:
Target = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv", index_col='row-col')
Target.head()

In [ ]:
# data = data.sample(frac =.25)

In [ ]:
# reference notebook
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='mean')
data_mean=data.copy()
data_mean[:] = imp.fit_transform(data)
for i in tqdm(Target.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    Target.loc[i, 'value'] = data_mean.loc[row, col]

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.2 | Basic statistics- Constant value</b></p>
</div>

In [ ]:
# imputing with a constant
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='constant')
data_constant=data.copy()
data_constant[:] = imp.fit_transform(data)
for i in tqdm(Target.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    Target.loc[i, 'value'] = data_constant.loc[row, col]

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.3 | Basic statistics- Most frequent</b></p>
</div>

In [ ]:
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='most_frequent')
data_frequent=data.copy()
data_frequent[:] = imp.fit_transform(data)
for i in tqdm(Target.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    Target.loc[i, 'value'] = data_frequent.loc[row, col]

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.1 | Advanced- K-Nearest Neighbor Imputation</b></p>
</div>

In [ ]:
# from sklearn.impute import KNNImputer
# data_knn = data.copy(deep=True)

# knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
# #tranforming individual column seperately
# for column in data_knn:   
#     data_knn[column] = knn_imputer.fit_transform(data_knn[[column]])
# for i in tqdm(Target.index):
#     row = int(i.split('-')[0])
#     col = i.split('-')[1]
#     Target.loc[i, 'value'] = data_knn.loc[row, col]

Notes: For quick EDA I have used uniform weights. We should try various options to see which one fit the best. This technique takes extremly longtime for large datasets. I have commented it out to minimze the runtime.

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.2 | Advanced- MICE</b></p>
</div>
Here Round-robin technique is used where the missing values are replaced as a functio nof other features. Mutiple regression is performed over random sample of the column data and then regression values are used to replace the missing values.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

data_MICE = data.copy(deep=True)
mice_imputer = IterativeImputer()
#tranforming individual column seperately
for column in data_MICE:   
    data_MICE[column] = mice_imputer.fit_transform(data_MICE[[column]])
for i in tqdm(Target.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    Target.loc[i, 'value'] = data_MICE.loc[row, col]

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.3 | Advanced- Regression</b></p>
</div>
 


An efficient way to handle missing values is predicting them using high performance regression techniques. This is a much more accurate solution to the problem. In this technique, one column is treated as a target at a time and try to predict the column's value using other columns. If the missing values are present only in the numerical columns, high performance regression models cand do the missing value predictions. For each column, we can take all the rows where the column value is present as the training dataset and the missing column values as the test dataset.

<img src="https://i.postimg.cc/90BdtTjq/imputer.gif">



### Here I have applied aforementioned regression technique to current data.

In [ ]:
from distutils.dir_util import copy_tree

from_dir = '../input/tpsjune2022-tabnet/'
to_dir = './'
copy_tree(from_dir, to_dir)
import os
os.rename(r'./submission_pytorch.csv',r'./submission.csv')



Full implimentation of above technique is shown in this public [notebook](https://www.kaggle.com/code/abdulravoofshaik/top-3-solution-lgbm-mean) which has resulted in very good score when compared with other techniques. Note that I have used LGBM, but you can try other techniques such as XGboost, RandomForest, Catboost....etc.

### Well, there is no single best way to handle missing values. One needs to experiment with different methods and then decide which method is best for a particular problem. 

## Work in progress

<div style="color:white;display:fill;border-radius:8px;
            background-color:#52BE80;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>5.0 | References</b></p>
</div>

https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009 <br>
https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4 <br>
https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook <br>
https://www.kaggle.com/code/residentmario/using-missingno-to-diagnose-data-sparsity/notebook <br>
https://www.analyticsvidhya.com/blog/2021/05/dealing-with-missing-values-in-python-a-complete-guide/ <br>
https://www.kaggle.com/competitions/tabular-playground-series-jun-2022/discussion/328369 <br>

